In [72]:
import numpy as np
from numpy.linalg import norm

In [57]:
class matrix_factorization():
    def __init__(self, data, number_of_features):
        self.data = data
        self.number_of_features = number_of_features        
        self.user_count = data.shape[0]
        self.item_count = data.shape[1]
        self.user_features = np.random.uniform(low=0.1, high=0.9, size=(self.user_count, self.number_of_features))
        self.item_features = np.random.uniform(low=0.1, high=0.9, size=(self.number_of_features, self.item_count))
    
    def mse(self):
        matrix_product = np.matmul(self.user_features, self.item_features)
        return np.sum((self.data - matrix_product)**2)
    
    def single_gradient(self, user_row, item_col, wrt_user_idx=None, wrt_item_idx=None):        
        
        if wrt_user_idx != None and wrt_item_idx != None:            
            return "To many elements!"
        elif wrt_user_idx == None and wrt_item_idx == None:            
            return "Insufficient elements!"
        
        else:
            u_row = self.user_features[user_row, :]
            i_col = self.item_features[:, item_col]
            reference_values = float(self.data[user_row, item_col])
            prediction = float(np.dot(u_row, i_col))
        
            if wrt_user_idx != None:
                row_element = float(i_col[wrt_user_idx])
                gradient = 2 * (reference_values - prediction) * row_element
            else:
                col_element = float(u_row[wrt_item_idx])
                gradient = 2 * (reference_values - prediction) * col_element               
            return gradient
    
    def user_feature_gradient(self, user_row, wrt_user_idx):
        summation = 0
        for col in range(0, self.item_count):
            summation += self.single_gradient(user_row=user_row, item_col=col, wrt_user_idx=wrt_user_idx)
        return summation / self.item_count
    
    def item_feature_gradient(self, item_col, wrt_item_idx):
        summation = 0
        for row in range(0, self.user_count):            
            summation += self.single_gradient(user_row=row, item_col=item_col, wrt_item_idx=wrt_item_idx)            
        return summation / self.user_count
    
    def update_user_features(self, learning_rate):
        for i in range(0, self.user_count):
            for j in range(self.number_of_features):
                self.user_features[i, j] += learning_rate * self.user_feature_gradient(user_row=i, wrt_user_idx=j)
    
    def update_item_features(self, learning_rate):
        for i in range(0, self.number_of_features):
            for j in range(0, self.item_count):                
                self.item_features[i, j] += learning_rate * self.item_feature_gradient(item_col=j, wrt_item_idx=i)
                
    def train(self, learning_rate=0.1, iterations=1000):
        for i in range(iterations):
            self.update_user_features(learning_rate)
            self.update_item_features(learning_rate)
            if (i % 50 == 0):
                print(self.mse())
        
    

## Treinando o modelo

In [92]:
data = np.array([[5,3,1], [1,3,5], [5,4,1], [0, 1, 0]])
data

array([[5, 3, 1],
       [1, 3, 5],
       [5, 4, 1],
       [0, 1, 0]])

In [93]:
model = matrix_factorization(data, 5)
model.train()

36.735759939219186
0.0010779167720836322
5.5043588483417025e-09
1.0381545357435983e-13
1.4961480423677698e-17
2.6112903615421708e-21
4.554615135253541e-25
7.652001514932373e-29
3.619704543330529e-30
3.3257246000873896e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30
3.572243632968956e-30


## Verificando as features

In [96]:
model.user_features

array([[ 1.76154154,  0.15923686,  0.47157139,  1.03974575,  0.99605132],
       [-0.34076851,  1.32005974,  1.94348245,  1.09540824,  0.39789041],
       [ 1.89492449,  0.82149761,  0.58567059,  0.27648202,  0.98641479],
       [-0.01896408,  0.75657779, -0.36078171, -0.01951881,  0.29335937]])

In [97]:
model.item_features

array([[ 1.94392167,  0.91216238, -0.53093523],
       [-0.08359942,  1.31159583,  0.61980744],
       [ 0.51968768,  0.6043597 ,  1.59840026],
       [ 0.33220069,  0.06710194,  0.67917381],
       [ 1.00249778,  0.83285465,  0.37813349]])

## Reconstruíndo a matriz original

In [98]:
np.dot(model.user_features, model.item_features)

array([[ 5.00000000e+00,  3.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  3.00000000e+00,  5.00000000e+00],
       [ 5.00000000e+00,  4.00000000e+00,  1.00000000e+00],
       [-2.41685828e-16,  1.00000000e+00,  3.05839025e-17]])

## Computando similaridade

In [99]:
def cosine_similarity(A,B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [100]:
cosine_similarity(model.user_features[0, :], model.user_features[2, :])

0.9052677440786913